In [1]:
import pandas as pd
import numpy as np

tabel = pd.read_csv('data_latih_opsi_2.csv', sep=',', header=None)

In [2]:
#15 bit

In [3]:
individu = 10
num_of_parents = 2

In [4]:
tabel = tabel.values

In [5]:
data = tabel[:,:]
# data

In [6]:
p = lambda f, i: (f[i] / sum(f))

In [7]:
def roulette(fit):
    r = np.random.uniform(0,1)
    indv = 0
    
    while r > 0:
        r -= p(fit,indv)
        
        indv += 1
    return indv-1

In [9]:
def fitness(dataset,rules):
    bnr = 0
    tot = len(dataset)
    for i, v in enumerate(dataset):
        has = tree(v[:-1], rules)
        if v[-1] == has:
            bnr += 1
        elif has == "unseen":
            tot -= 1
    return bnr/tot

In [10]:
def oneorzero():
    return int(np.random.rand() > .5)

In [11]:
def encode(decoded):
    bit1, bit2, bit3, bit4, bit5 = np.zeros(3), np.zeros(4), np.zeros(4), np.zeros(3), np.zeros(1)
    bit1[decoded[0]] = 1
    bit2[decoded[1]] = 1
    bit3[decoded[2]] = 1
    bit4[decoded[3]] = 1
    bit5[0] = decoded[4]
    return np.concatenate([bit1, bit2, bit3, bit4, bit5], axis=0)

In [12]:
def decode(encoded):
    bits = [None] * 5
    bits[0], bits[1], bits[2], bits[3], bits[4] = encoded[:3], encoded[3:7], encoded[7:11], encoded[11:14], encoded[-1]
    
    nbits = []
    for i, v in enumerate(bits[:-1]):
        bitz = []
        for j, k in enumerate(v):
            if k == 1:
                bitz.append(j)
        nbits.append(bitz)
    nbits.append([bits[-1]])
    return nbits

In [13]:
def initializeRule():
    krom = [None] * 5
    krom[0], krom[1], krom[2], krom[3], krom[4] = np.zeros(3), np.zeros(4), np.zeros(4), np.zeros(3), np.zeros(1)
    
    for i, v in enumerate(krom[:-1]):
        while sum(krom[i]) == 0:
            for j, k in enumerate(v):
                krom[i][j] = oneorzero()
    krom[4] = np.array([oneorzero()])
    nkrom = np.concatenate(krom)
    return nkrom

In [14]:
def initialize(individu, brprule):
    population = []
    for i in range(individu):
        rules = []
        for j in range(brprule):
            rules.append(initializeRule())
        population.append(np.concatenate(rules, axis=0))
    return population

In [15]:
def mutation(individu):
    r = np.random.uniform(0,1)
    if r < 0.01: #1% chance
        pos = np.random.randint(len(individu))
        individu[pos] = 1 if individu[pos] == 0 else 0
    return individu

In [16]:
def crossover(parents):
    '''Sementara crossover yg gak dinamis, susah soalnya bikin yg dinamis wkwkwk'''
    leng = len(parents[0])
    r1 = np.random.randint(leng-2) + 1
    r2 = np.random.randint(leng-2) + 1
    while r1 == r2:
        r1 = np.random.randint(leng-2) + 1
        r2 = np.random.randint(leng-2) + 1
    mn = min(r1, r2)
    mx = max(r1, r2)
#     print(mn, mx)
    new_parents = [None] * 2
    new_parents[0] = np.concatenate([parents[0][:mn], parents[1][mn:mx], parents[0][mx:]])
    new_parents[1] = np.concatenate([parents[1][:mn], parents[0][mn:mx], parents[1][mx:]])
                                    
    return new_parents

In [17]:
def tree(test, rules):
    realdata = test
#     dtree_rules = rules

    bits = [None] * 5
    bits[0], bits[1], bits[2], bits[3], bits[4] = [], [], [], [], []
#     print(rules)
    dr = np.array_split(rules, len(rules)/15)
    dtree_rules = [decode(i) for i in dr]
#     for i, v in enumerate(dr):
#         print(v)
#         print(decode(v))
#     print(dtree_rules)
    ret = 'unseen'
    for p, l in enumerate(dtree_rules):
        asem = []
        for i, v in enumerate(l[:-1]):
            sem = []
#             print(v)
            for j, k in enumerate(v):
                
                
                sem.append(realdata[i] == k)
            v = any(sem)
            asem.append(v)
        if all(asem) == True:
            ret = l[-1][0]
            break
#     print("ret",ret)
    
    return ret
    

In [18]:
generasi = 300

indvs = np.array(initialize(individu,60))
# print(indvs)
for gen in range(generasi):
    fitnesses = []
    for i, v in enumerate(indvs):
        fitnesses.append(fitness(data, v))

    parents = []
    for i in range(num_of_parents):
        parents.append(np.array(indvs[roulette(fitnesses)]))
    parents = np.array(parents)
    # print(parents)
#     print(parents.shape)
#     print(indvs[0:2, :])
    offsprings = np.array(crossover(parents))
    mutated_offspring = []
    for i, offspring in enumerate(offsprings):
        mutated_offspring.append(mutation(offspring))
    mutated_offspring = np.array(mutated_offspring)
#     print(parents.shape)
#     print(offsprings.shape)
#     print(mutated_offspring.shape)
    indvs[0:parents.shape[0]] = parents
    indvs[-parents.shape[0]:] = mutated_offspring
    
#     print(indvs.shape)
    n_fitnesses = [fitness(data, i) for i in indvs]
    bsf = np.max(n_fitnesses)
    idx = np.argmax([fitness(data, i) for i in indvs],axis=0)
#     print('all fitnesses', n_fitnesses)
    print("best so far accuracy:",bsf,"at index:", idx)

best so far accuracy: 0.6 at index: 2
best so far accuracy: 0.6 at index: 0
best so far accuracy: 0.6 at index: 0
best so far accuracy: 0.6 at index: 2
best so far accuracy: 0.6 at index: 2
best so far accuracy: 0.6 at index: 2
best so far accuracy: 0.6 at index: 2
best so far accuracy: 0.6 at index: 2
best so far accuracy: 0.6 at index: 2
best so far accuracy: 0.6 at index: 2
best so far accuracy: 0.6 at index: 2
best so far accuracy: 0.6 at index: 2
best so far accuracy: 0.6 at index: 2
best so far accuracy: 0.6 at index: 1
best so far accuracy: 0.6 at index: 1
best so far accuracy: 0.6125 at index: 8
best so far accuracy: 0.6 at index: 2
best so far accuracy: 0.6 at index: 2
best so far accuracy: 0.6 at index: 0
best so far accuracy: 0.6 at index: 0
best so far accuracy: 0.6 at index: 2
best so far accuracy: 0.6 at index: 2
best so far accuracy: 0.6 at index: 2
best so far accuracy: 0.65 at index: 9
best so far accuracy: 0.6 at index: 2
best so far accuracy: 0.6 at index: 0
best so 

KeyboardInterrupt: 